#### Descargando la data 

Descargando los datos para el entrenamiento de los modelos.

#### Módulos

In [1]:
# Módulos
import ee
import folium
import geehydro

In [3]:
# Inicio de API
ee.Initialize()

#### Función generateGreedForGeometry

In [5]:
#funcion para la generacion de grillas
def generateGreedForGeometry(geom,dx,dy):
    """
    Función para generar grillas partiendo de una zona geográfica.
    """

    bounds = ee.Geometry(geom).bounds()
    coords = ee.List(bounds.coordinates().get(0))
    ll = ee.List(coords.get(0))
    ur = ee.List(coords.get(2))
    xmin = ll.get(0)
    xmax = ur.get(0)
    ymin = ll.get(1)
    ymax = ur.get(1)

    xx = ee.List.sequence(xmin, xmax, dx)
    yy = ee.List.sequence(ymin, ymax, dy)

    #************************************
    def contYYa(y,x,dx,dy):

        x1 = ee.Number(x)
        x2 = ee.Number(x).add(ee.Number(dx))
        y1 = ee.Number(y)
        y2 = ee.Number(y).add(ee.Number(dy))

        coords = ee.List([x1, y1, x2, y2])
        rect = ee.Algorithms.GeometryConstructors.Rectangle(coords)

        return ee.Feature(rect)

    def contYY(x):
        return yy.map(lambda yi: contYYa(y=yi,x=x,dx=dx,dy=dy) )
    #************************************

    cells = xx.map(contYY).flatten() 

    return ee.FeatureCollection(cells).filterBounds(geom)#.map(lambda c: ee.Feature(c).intersection(geom) )

#### Ejemplo

In [172]:
#----------------------------------------------
# Seleccion del parque
#----------------------------------------------

# region
polygonCollection = 'WCMC/WDPA/current/polygons'
polygonName = 'Cerro Saroche'#'Canaima National Park'#'Guatopo'

# poligono
polygon = ee.FeatureCollection(polygonCollection) \
            .filter(ee.Filter.eq('NAME', polygonName))


# geometria
state = polygon.geometry()
dx = 0.0288
dy = 0.0288

# creando grilla
greed = generateGreedForGeometry(state,dx,dy)
centroide = state.centroid().coordinates().getInfo()[::-1]

def getCentroid(feature):
    keepProperties = ['name', 'huc6', 'tnmid', 'areasqkm']
    centroid = feature.geometry().centroid()
    return ee.Feature(centroid).copyProperties(feature, keepProperties)

#centroides
centroids = greed.map(getCentroid)

print('área del parque Km2: ', 0.1*polygon.first().geometry().area().divide(100*100).getInfo() ) 
print('área de cuadricula Km2: ', 0.1*greed.first().geometry().area().divide(100*100).getInfo() )
print('número de centroides: ', centroids.size().getInfo())

área del parque Km2:  3629.434345090513
área de cuadricula Km2:  100.9365181750428
número de centroides:  74


#### data ECMWF/ERA5/MONTHLY y CGIAR/SRTM90_V4

In [178]:
# Elevacion
elevation = ee.Image('CGIAR/SRTM90_V4').select('elevation')
#slope = ee.Terrain.slope(elevation)

# Precipitacion total
ECMWF = ee.ImageCollection("ECMWF/ERA5/MONTHLY")
#slope = ee.Terrain.slope(elevation)

# calculando el punto con la elevacion
reducer = ee.Reducer.first()
centroids_elevation = elevation.reduceRegions( ee.FeatureCollection(centroids) , reducer.setOutputs(['elevation']), 90)

In [324]:
def ECMWF_data(imagen_collection = ECMWF,
                date = '2015-12-01',
                points = centroids,
                imagen_elevation = centroids_elevation):

    reducer = ee.Reducer.first()
    centroids_ECMWF= imagen_collection\
                        .filterDate(date)\
                        .map(lambda x: x.reduceRegions(imagen_elevation,reducer.setOutputs(['total_precipitation']),2))

    lista = centroids_ECMWF.first().getInfo()

    if lista is None:
        return None

    return list(map(lambda x,y:
                    {'date': y,
                    'id':int(x.get('id')),
                    'longitude':x.get('geometry').get('coordinates')[0],
                    'latitude':x.get('geometry').get('coordinates')[1],
                    'dewpoint_2m_temperature': x.get('properties').get('dewpoint_2m_temperature'),
                    'elevation': x.get('properties').get('elevation'),
                    'maximum_2m_air_temperature': x.get('properties').get('maximum_2m_air_temperature'),
                    'mean_2m_air_temperature': x.get('properties').get('mean_2m_air_temperature'),
                    'mean_sea_level_pressure': x.get('properties').get('mean_sea_level_pressure'),
                    'minimum_2m_air_temperature': x.get('properties').get('minimum_2m_air_temperature'),
                    'surface_pressure': x.get('properties').get('surface_pressure'),
                    'total_precipitation': x.get('properties').get('total_precipitation'),
                    'u_component_of_wind_10m': x.get('properties').get('u_component_of_wind_10m'),
                    'v_component_of_wind_10m': x.get('properties').get('v_component_of_wind_10m')
                    },
                    lista.get('features'),
                    [lista.get('properties').get('system:index') for x in range(len(lista.get('features'))) ]
                    ))

#ECMWF_data()

In [343]:
import pandas as pd
import numpy as np

# rango de meses
date_start="2010-01-01"
date_end="2010-03-01"

# consultando la data
data = np.array( list(filter(None,
                    list(map(lambda x: ECMWF_data(date=x)
                                    ,pd.date_range(start=date_start,end=date_end, closed='right',freq='M').strftime('%Y-%m').to_list()
                                    ))
                )))

data = data.reshape((
                    1,
                    data.shape[0]*data.shape[1]
                    )).tolist()[0]

data = pd.DataFrame(data)
data['year'] = pd.to_datetime(data['date'],format='%Y%m').dt.year
data['month'] = pd.to_datetime(data['date'],format='%Y%m').dt.month
data['date'] = data['date'].apply(int)

data.sort_values(['date','id'],ascending=True).head(30)

,date,id,longitude,latitude,dewpoint_2m_temperature,elevation,maximum_2m_air_temperature,mean_2m_air_temperature,mean_sea_level_pressure,minimum_2m_air_temperature,surface_pressure,total_precipitation,u_component_of_wind_10m,v_component_of_wind_10m,year,month
0,201001,4,-69.862964,10.191936,289.937256,494,310.467957,300.634796,101112.992188,293.052795,95308.664062,0.000668,-3.618990,-0.772696,2010,1
1,201001,5,-69.862964,10.220736,289.937256,734,310.467957,300.634796,101112.992188,293.052795,95308.664062,0.000668,-3.618990,-0.772696,2010,1
2,201001,6,-69.862964,10.249536,289.937256,538,310.467957,300.634796,101112.992188,293.052795,95308.664062,0.000668,-3.618990,-0.772696,2010,1
3,201001,13,-69.834164,10.191936,289.937256,512,310.467957,300.634796,101112.992188,293.052795,95308.664062,0.000668,-3.618990,-0.772696,2010,1
4,201001,14,-69.834164,10.220736,289.937256,618,310.467957,300.634796,101112.992188,293.052795,95308.664062,0.000668,-3.618990,-0.772696,2010,1
5,201001,15,-69.834164,10.249536,289.937256,706,310.467957,300.634796,101112.992188,293.052795,95308.664062,0.000668,-3.618990,-0.772696,2010,1
6,201001,16,-69.834164,10.278336,289.905151,507,308.866150,298.711700,101191.000000,291.120056,93795.914062,0.002523,-3.175932,-0.149812,2010,1
7,201001,22,-69.805364,10.191936,289.937256,499,310.467957,300.634796,101112.992188,293.052795,95308.664062,0.000668,-3.618990,-0.772696,2010,1
8,201001,23,-69.805364,10.220736,289.937256,601,310.467957,300.634796,101112.992188,293.052795,95308.664062,0.000668,-3.618990,-0.772696,2010,1
9,201001,24,-69.805364,10.249536,289.937256,969,310.467957,300.634796,101112.992188,293.052795,95308.664062,0.000668,-3.618990,-0.772696,2010,1


In [141]:
# extract points from DEM
fc = ee.FeatureCollection(centroids)
reducer = ee.Reducer.first()
centroids_elevation = elevation.reduceRegions(fc, reducer.setOutputs(['elevation']), 90)
centroids_ECMWF= precipitacion_total.map(lambda x: x.reduceRegions(centroids_elevation,reducer.setOutputs(['total_precipitation']),2))
#centroids_precipitacion_total = precipitacion_total.map(lambda x: x.reduceRegions(centroids_elevation, reducer.setOutputs(['total_precipitation']), 2))
#centroids_temperatura_media = precipitacion_total.map(lambda x: x.reduceRegions(centroids_elevation, reducer.setOutputs(['mean_2m_air_temperature']), 2))

In [160]:
task = ee.batch.Export.table.toDrive(**{
  'collection': centroids_ECMWF,
  'description': 'exportTableExample',
  'fileFormat': 'CSV'
})

In [14]:
task = ee.batch.Export.table.toDrive(data, 'pointsDataExtract', 'FOLDER', 'nameOfCSV')
task.start()

#### Información Indices de vegetacion

In [8]:
def getQABits(image, start, end, newName):
    """
    Funcion para establecer la calidad de las bandas
    """

    import math

    pattern = 0
    for i in range(start,end+1):
      pattern += math.pow(2, i)
    #Return a single band  image of the extracted QA bits, giving the band
    #a new name.
    return image.select([0], [newName])\
                  .bitwiseAnd(pattern)\
                  .rightShift(start)

#### MOD13Q1

In [91]:
# MODIS MOD13Q1
modis = ee.ImageCollection('MODIS/006/MOD13Q1')\
             .filterDate(ee.DateRange(date_ini,date_end))\
             .map(lambda x: x.clip(polygon))


# Filtros de calidad -----------------------------------------
# def modis_bit_0(image):

#   qscore = getQABits(image.select('SummaryQA'), 0, 1, 'quality')\
#               .expression("b(0) == 2 || b(0) == 3" )

#   return image.updateMask(qscore.Not())

# #Select the QA band.
# def modis_bit_1(image):

#   qscore = getQABits(image.select('SummaryQA'), 2, 5, 'usefulness')\
#               .expression("b(0) == 2 || b(0) == 4 || b(0) == 8 || b(0) == 9 || b(0) == 10 || b(0) == 12 || b(0) == 13 || b(0) == 14 || b(0) == 15" )

#   return image.updateMask(qscore.Not())

# #Select the QA band.
# def modis_bit_2(image):

#   qscore = getQABits(image.select('SummaryQA'), 13, 14, 'Land/water')\
#               .expression("b(0) == 0 || b(0) == 1 || b(0) == 2 || b(0) == 3 || b(0) == 4 || b(0) == 5 || b(0) == 6 || b(0) == 7" )

#   return image.updateMask(qscore.Not())

# #Select the QA band.
# def modis_bit_3(image):

#   qscore = getQABits(image.select('SummaryQA'), 14, 14, 'snow/ice')\
#               .expression("b(0) == 1" )

#   return image.updateMask(qscore.Not())

# #Select the QA band.
# def modis_bit_4(image):

#   qscore = getQABits(image.select('DetailedQA'), 15, 15, 'shadow')\
#               .expression("b(0) == 1" )

#   return image.updateMask(qscore.Not())

# Filtros de calidad -----------------------------------------


def scale_factor(image):
    # scale factor for the MODIS MOD13Q1 product
    return image.multiply(0.0001).copyProperties(image,['system:time_start'])



In [92]:
# select EVI and NDVI
modis_qa = modis#.map(modis_bit_0)
                # .map(modis_bit_1)\
                # .map(modis_bit_2)\
                # .map(modis_bit_3)\
                # .map(modis_bit_4)
                
evi = modis_qa.select('EVI').map(scale_factor).mean()
ndvi = modis_qa.select('NDVI').map(scale_factor).mean()

#### SRTM Digital Elevation Data Version 4

In [93]:
# Elevacion
elevation = ee.Image('CGIAR/SRTM90_V4').select('elevation')
slope = ee.Terrain.slope(elevation).clip(polygon)


MCD64A1.006 MODIS Burned Area Monthly Global 500m

In [94]:
fire = ee.ImageCollection('MODIS/006/MCD64A1')\
            .filter(ee.Filter.date(date_ini,date_end))\
            .map(lambda x:x.clip(polygon))



# Filtros de calidad -----------------------------------------
def bit_0(image):

  qscore = getQABits(image.select('QA'), 0, 0, 'Land_water')\
              .expression("b(0) == 0")

  return image.updateMask(qscore.Not())

#Select the QA band.
def bit_1(image):

  qscore = getQABits(image.select('QA'), 1, 1, 'Valid_data_flag')\
              .expression("b(0) == 0")

  return image.updateMask(qscore.Not())

# Filtros de calidad -----------------------------------------

burnedArea = fire.map(bit_0)\
                 .map(bit_1)\
                 .select('BurnDate')

#### Mapa

In [95]:
# mean NDVI in the Xingu Park
Map = folium.Map(location=centroide,zoom_start=11,control_scale =True,width='100%',height='100%')

Map.addLayer(slope, {min: 0, max: 60}, 'Slope')
Map.addLayer(ndvi
            ,name='NDVI'
            ,vis_params={'min': 0,
                         'max': 1,
                         'palette': ['953302', '62D759','499443']}) 
Map.addLayer(evi
            ,name='EVI'
            ,vis_params={'min': 0,
                         'max': 1,
                         'palette': ['953302', '62D759','499443']}) 
Map.addLayer(burnedArea, {'min': 30.0,
                        'max': 341.0,
                        'palette': ['4e0400', '951003', 'c61503', 'ff1901'],
                        }, 'BurnedArea')
                        
Map.addLayer(greed,{'color':'#FFFFFF80'}, 'Grids')
Map.addLayer(centroids,{'color':'#000000'}, 'Centroids')

Map.setControlVisibility(layerControl=True,fullscreenControl=True,latLngPopup=True) # Layer control
Map

#### Series temporales

##### Índices de vegetación

In [96]:
initDate = '2015-01-01' # Fecha de inicio
endDate = '2020-05-01' # Fecha final

In [97]:
#Filtros de calidad -----------------------------------------
def modis_bit_0(image):

  qscore = getQABits(image.select('SummaryQA'), 0, 1, 'quality')\
              .expression("b(0) == 1 || b(0) == 2 || b(0) == 3" )

  return image.updateMask(qscore.Not())

#Select the QA band.
def modis_bit_1(image):

  qscore = getQABits(image.select('DetailedQA'), 2, 5, 'usefulness')\
              .expression("b(0) == 13 || b(0) == 14 || b(0) == 15" )

  return image.updateMask(qscore.Not())

#Select the QA band.
def modis_bit_2(image):

  qscore = getQABits(image.select('DetailedQA'), 11, 13, 'Land/water')\
              .expression("b(0) == 0 || b(0) == 1 || b(0) == 2 || b(0) == 3 || b(0) == 4 || b(0) == 5 || b(0) == 6 || b(0) == 7" )

  return image.updateMask(qscore.Not())

#Select the QA band.
def modis_bit_3(image):

  qscore = getQABits(image.select('DetailedQA'), 14, 14, 'snow/ice')\
              .expression("b(0) == 1" )

  return image.updateMask(qscore.Not())

#Select the QA band.
def modis_bit_4(image):

  qscore = getQABits(image.select('DetailedQA'), 15, 15, 'shadow')\
              .expression("b(0) == 1" )

  return image.updateMask(qscore.Not())

#Filtros de calidad -----------------------------------------


def scale_factor(image):
    # scale factor for the MODIS MOD13Q1 product
    return image.multiply(0.0001).copyProperties(image,['system:time_start'])

In [98]:
MOD13Q1Collection = ee.ImageCollection('MODIS/006/MOD13Q1')\
                      .filterDate(ee.DateRange(initDate, endDate))\
                      .map(lambda x: x.clip(polygon))\
                      .map(modis_bit_0)\
                      .map(scale_factor)
                    #   .map(modis_bit_1)\
                    #   .map(modis_bit_2)\
                    #   .map(modis_bit_3)\
                    #   .map(modis_bit_4)

In [99]:
from ipygee import chart

# Media del EVI en polígono
EVI_serie = chart.Image.series(**{'imageCollection': MOD13Q1Collection,
                                  'region': polygon,
                                  'reducer': 'mean',
                                  'scale': 1000,
                                  'xProperty': 'system:time_start',
                                  'bands':'EVI'}).dataframe
# Media del NDVI en polígono
NDVI_serie = chart.Image.series(**{'imageCollection': MOD13Q1Collection,
                                    'region': polygon,
                                    'reducer': 'mean',
                                    'scale': 1000,
                                    'xProperty': 'system:time_start',
                                    'bands':'NDVI'}).dataframe

In [100]:
# agregando el tiempo
EVI_serie['date'] = EVI_serie.index 
NDVI_serie['date'] = NDVI_serie.index 

In [101]:
import plotly.graph_objects as go

# Create traces
fig_evi = go.Figure()
fig_evi.add_trace(go.Scatter(x=EVI_serie.date.tolist(), y=EVI_serie.EVI.tolist(),
                    mode='lines+markers',
                    name='EVI Time Serie',
                    line=dict(color='#78D759', width=4)
                    ))

# Edit the layout
fig_evi.update_layout(title='Serie de tiempo EVI',
                      xaxis_title='Mes',
                      yaxis_title='Índice de Vegetación')

fig_evi.show()

In [102]:
# Create traces
fig_ndvi = go.Figure()
fig_ndvi.add_trace(go.Scatter(
                    x=NDVI_serie.date.tolist(),
                    y=NDVI_serie.NDVI.tolist(),
                    mode='lines+markers',
                    name='NDVI Time Serie',
                    line=dict(color='#4E8C39', width=4)
                    ))
# Edit the layout
fig_ndvi.update_layout(title='Serie de tiempo NDVI',
                       xaxis_title='Mes',
                       yaxis_title='Índice de Vegetación')

fig_ndvi.show()